# Improving and combining functions

### Outline

* Lambda functions

* Composing and applying functions

* Curried functions and partial application

* Higher order functions

## Lambda functions

Back in the 1930s, a young American mathematician Alonzo Church asked himself the question: *what does it means to "calculate" something?* 

The result of his reflections was a system that formalized the concept of "computation", and he called this system "lambda calculus" (named after the Greek letter $\lambda$). 

At the heart of this system lies the lambda function, which in a sense can be considered "mother of functional programming" in general and Haskell in particular. 

### What is a lambda function?

The idea of ​​lambda functions is so useful because it is extremely simple: **lambda function is an anonymous function**.

Here's how a squaring function $f(x)=x^2$ looks like in Haskell:

```haskell
\x -> x * x
```

The backslash `\` at the beginning denotes the beginning of lambda function. We can think of the backslash `\` in the definition of lambda function as an upper half of letter $\lambda.$

### Is lambda function a function?

Any lambda function is *the simplest form of a function*. 

* We removed a signature of a function.

* And we removed its name.

* Only the necessary minimum is left:

    * parameter names
    
    * and a body of function.

### What is a difference between lambda function and an ordinary function?

* An ordinary function starts with a signature/definition, and it is used somewhere by its name.

* A lambda function doesn't need to have signature and a name, and it can be applied immediately.

Here is an example:

In [ ]:
(\x -> x * x) 5

The squaring function is defined and is immediately applied to the parameter `5`.

### Structure

Every lambda function consists of 4 ingredients.

* A backslash `\` meaning the beginning of a lambda function.

* A list of parameters separated by *spaces*.

* An arrow `->` that separates parameters from the body of a function.

* A expression/formula/... of our function.

Number multiplication can be given by lambda function of two parameters:

In [ ]:
(\x y -> x * y) 10 4

**Remark!** Lambda functions can be given *names* and we can call them as ordinary functions (for instance, using `where` construction).

```haskell
rectangleArea :: Double -> Double -> Double
rectangleArea width length = mult width length
    where mult = \x y -> x * y
```

### Type of lambda function

Any function has a type, so does a lambda function. So, what is a type of lambda function?

We've used a lambda function `mult` above in the `where` construction, hence it has a type. It is a polymorphic function:

```haskell
mul :: a -> a -> a
```

Since any function is characterized by its type in the same way as any other data, we make a conclusion 

                **Functions can be manipulated like data!**

For example, you can create a list of functions:

```haskell
helloFunction :: String -> String
helloFunction msg = (head functions) msg
    where
        functions = [ \x -> x ++ " world!"
                    , \x -> x ++ " everyone!"
                    ]
```

The list `functions` has two functions. Two lambda expressions give these two functions, but until the moment they are applied, they do nothing and are useless. 

But when we apply the head function to this list, we get the first element of the list, that is, the first function. And having received, immediately apply this function to the string `msg`.

The type of a list `functions` is

```haskell
[String -> String]
```

So it is a list of functions with of one argument of type `String` that return a value of type `String`.

### Local functions

Since there are actually no differences between lambda functions and ordinary functions, and functions are data, we can create functions locally for other functions.

Let's look at a naive function that checks if a given string is an email by checking whether it contains character `@` and there is a text (mailbox name) before it.

```haskell
validEmail :: String -> Bool
validEmail email = containsAtSign email && hasDot email
    where
        containsAtSign e = '@' `elem` e
        hasName e        = '@' /= head e
```

A body of this function has two boolean expressions with `&&` operator between them. In the `where` section there are defined two functions, that exist only for the main expression of the `validEmail` function. 

Using lambda functions the above code becomes:

```haskell
validEmail :: String -> Bool
validEmail email = containsAtSign email && hasDot email
    where
        containsAtSign = \e -> '@' `elem` e
        hasName        = \e -> '@' /= head e
```

Now the expressions `containsAtSign` and `hasName` are lambda functions of one argument. In this case, we do not specify the type of these expressions. 

However, if necessary, you can specify the type:

```haskell
containsAtSign = (\e -> '@' `elem` e) :: String -> Bool
```

<div class="alert alert-block alert-warning">
<b>Warning!</b> A lambda function must be parenthesized, if you want to declare its type, otherwise the type indication refers not to a function in general, but to its argument.
</div>

```haskell
-- Argument e has type String -> Bool, not a function!
containsAtSign = \e -> '@' `elem` e :: String -> Bool
```

## Composing functions

Now we'll explain how to compose functions together and how to get rid of (or just reduce) number of parentheses in the body of the function.

### Example with parentheses

Recall our old function `checkLocalHost` that checks if the argument is a localhost or not and reports the user the result.

```haskell
checkLocalhost :: String -> String
checkLocalhost ip =
    if ip == "127.0.0.1"
        then "It’s a localhost!"
        else "No, it's not a localhost."
```

A function `quadrToIP` takes a quadruple of integers and makes IP address.

```haskell
quadrToIP :: Int -> Int -> Int -> Int -> String
quadrToIP a b c d = show a ++ "." ++ show b ++ "." ++ show c ++ "." ++ show d
```

The application of these functions looks as

In [ ]:
checkLocalhost (quadrToIP 127 0 0 1)

Two functions are **composed** here, `checkLocalhost` and `quadrToIP`, because `quadrToIP` returns `String` and `checkLocalhost` function has `String` at the input. 

Schematically, this can be represented as follows:

$$
    \boxed{\mathrm{(Int, Int, Int, Int)}}
        \xrightarrow{\mathrm{~~~~~~quadrToIP~~~~~~}}
    \boxed{\mathrm{String}}
        \xrightarrow{~~~~~~\mathrm{checkLocalhost}~~~~~~}
    \boxed{\mathrm{String}}
$$

The way of composing functions above uses parenthesis. But what will happen if we need to compose three, four or more functions? The number of parenthesis grows and our code becomes less readable. There is another way how to compose functions.

### Function composition and application operator

The composition `checkLocalhost (quadrToIP 127 0 0 1)` can be written as

```haskell
checkLocalhost . quadrToIP $ 127 0 0 1
```

We have two new standard operators that save us from extra parentheses and making our code easier.

* The function **composition operator** given by a dot character `.`

* The **application operator** `$`

The composition operator combines two functions together (or **composes** them). When we write

```haskell
checkLocalhost . quadrToIP
```

instead of two functions `checkLocalhost` and `quadrToIP` we obtain a new one `checkLocalhost . quadrToIP` called their **composition**. Schematically, this can be represented as follows:

$$
    \boxed{\mathrm{(Int, Int, Int, Int)}}
        \xrightarrow{\mathrm{~~~~~~quadrToIP~~~~~~}}
    \boxed{\mathrm{String}}
        \xrightarrow{~~~~~~\mathrm{checkLocalhost}~~~~~~}
    \boxed{\mathrm{String}}
    ~~~~~
    =
    ~~~~~
    \boxed{\mathrm{(Int, Int, Int, Int)}}
        \xrightarrow{\mathrm{~~~~~~checkLocalhost~.~quadrToIP~~~~~~}}
    \boxed{\mathrm{String}}
$$


This is the essence of composition: **we take two functions and combine them into the third one**. 

Any function `f :: a -> b` can be thought as a path named `f` from a point `a` on a map to a point `b`. Hence, the composition of `f :: a -> b` and `g :: b -> c` is a path `g . f :: a -> c` that goes from `a` to `c` passing through `b`. 

Now it becomes clear why the arrow `->` was chosen as a separator between inputs and outputs of a function.

The application operator `$` is simply an evaluation of a function at a given value. Without `$` our code will look

```haskell
(checkLocalhost . quadrToIP) 127 0 0 1
```

But we wanted to get rid of the parentheses, and here they are again! For this we need the application operator `$`. Its scheme is simple:

```haskell
FUNCTION $ ARGUMENT
```

where `FUNCTION` is applied to `ARGUMENT`.

#### Remarks about composition and application

* Composition is *right associative*: the function on the right is applied first, and then the one on the left.

* If there are two functions `f :: a -> b` and `g :: b -> c` then their composition as a function can be written without the application operator. For instance, you can write the composition of `checkLocalhost` and `quadrToIP` as a new function `isQuadrLocalhost` using arguments

```haskell
isQuadrLocalhost :: Int -> Int -> Int -> Int -> String
isQuadrLocalhost a b c d = checkLocalhost . quadrToIP $ a b c d
```

or without them

```haskell
isQuadrLocalhost :: Int -> Int -> Int -> Int -> String
isQuadrLocalhost = checkLocalhost . quadrToIP 
```

* The composition operator is defined just like any other function. Here is its definition:

```haskell
(.) f g = \x -> f (g x)
```


The composition operator is applied to two functions, because we have already found out that functions can be operated on as a data. Hence, we can pass a function as an argument to another function! The composition operator takes two functions as an input, and then just gives us lambda function, inside which there is an ordinary sequential call of these two functions through parentheses.

* The application operator for a composition can be applied before the argument or after the first evaluation. You can write a code

```haskell
checkLocalhost . quadrToIP $ 127 0 0 1
```

in the other alternative form

```haskell
checkLocalhost $ quadrToIP 127 0 0 1
```

The reason is that it doesn't matter whether you are evaluating the whole composition at a given argument, or the second function at a value of the first function:

$$
    (g~.~f) x = g(f(x))
$$

## Higher order functions

As we've already learned, functions are data in Haskell. The function that operates on other functions as arguments and / or returns a function as a result is called a **higher order function**. 

Thus, the function composition operator is a higher order function, because

* it takes functions as arguments and

* it returns another function (in the form of lambda function) as a result of its application. 

Using functions as arguments is an extremely common practice in Haskell.

### Curried functions

Every function in Haskell officially only takes one parameter. So how is it possible that there are functions that take more than one parameter? 

All the functions that accepted several parameters so far have been **curried functions** (the name *curry* derives from the person who popularized the idea: Haskell Curry). 

Let's understand it on an example of the `max` function: it looks like it takes two parameters and returns the one that's bigger. 

* If we need to compute `max 4 5` then we first create a function that takes a parameter and returns either 4 or that parameter, depending on which is bigger. 

* Then, 5 is applied to that function and that function produces our desired result. 

The following two calls are equivalent:

In [ ]:
max 4 5
(max 4) 5

Putting a space between two things is simply **function application**. The **space** is sort of like an operator and it is similar to application operator `$`. 

`max` function is polymorphic and its type is `max :: a -> a -> a`, which can also be written as `max :: a -> (a -> a)`. 

That could be read as: `max` function takes an `a` and returns (that's the `->`) a function that takes an `a` and returns an `a`. That's why the return type and the parameters of functions are all simply separated with arrows.

The first version `max :: a -> a -> a` is an **uncurried function**, while the second one `max :: a -> (a -> a)` is **curried**.

Let's consider another example of the addition function.

```haskell
add :: Int -> Int -> Int
add x y =  x + y
```

This is an example of a curried function. To get the effect of an uncurried function, we could use a tuple, as in:

```haskell
add :: (Int, Int) -> Int
add (x,y) = x + y
```

But then we see that this version of add is really just a function of one argument! 

An application of add has the form `add num1 num2`, and is equivalent to `(add e1) e2`, since function application associates to the left. 

The type of add is `Int -> Int -> Int`, which is equivalent to `Int -> (Int -> Int)`, hence `->` associates to the right. 

### Partial application

In other words, curring is applying a function `f :: a -> b -> c` to one argument which yields a new function, that is then applied to the second argument. 

But what will happen if we'll pass it not two arguments, but only to one? In this case, there will be one "magical" transformation, called a partial application of a function. Thus, the application of a function is partial is when there are less arguments than expected.

Using partial application (calling functions with too few parameters, if you will) is a neat way to create functions on the fly so we can pass them to another function or to seed them with some data.

Thus, `(max 4) 5` is a function application of a partial function 

```haskell
\x -> max 4 x
```

to an argument 5. 

<div class="alert alert-block alert-warning">
<b>Warnings:</b> 
From now on, we'll say that functions take several parameters despite each function is taking only one parameter and returning partially applied functions until we reach a function that returns a solid value.
</div>

 So for simplicity's sake, we'll say that `a -> a -> a` takes two parameters, even though we know what's really going on.

#### Is there a difference between the application operator $ and a function application given by space?

The `$` operator is an infix operator for function application and its signature is

```haskell
($) :: (a -> b) -> a -> b
```

So, given an `a -> b` function and an `a` to apply it to, it gives us a `b`.

On the level of application both `$` and space give the same results:

In [ ]:
sqrt 4
sqrt $ 4

However, if we try the following code, the results will be surprising.

In [ ]:
sqrt 4 + 12
sqrt $ 4 + 12

In the first case the square root is applied first to 4, and only then 12 is added, while in the second one the square root is taken from the sum of 4 and 12. The reason is that the application operator has the lowest priority, and function application - the highest one. Moreover, one of them is right and another one is left associative.

* `$` is an infix operator with right associativity and has the lowest priority possible (Haskell takes everything to its left (that doesn't include another $) and everything to its right (likewise), and effectively puts both sides in parentheses).

* The function application (via space) is left associative and has the highest priority possible (Haskell evaluates each function application first and perform other actions afterwards). 

So the role of the `$` operator is to give us function application with a different – opposite, really – associativity and priority.

What that means is that you usually see the `$` where standard function application wouldn’t have the necessary associativity and priority for the context. And what that means is you usually see it used instead of parentheses to associate things that otherwise wouldn’t.